In [5]:
import pandas as pd
import os
import sys

In [6]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [7]:
BW_PROJECT = 'metallican' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	cutoff310

# Regioinvent

In [8]:
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append(r"C:\Users\mp_ma\OneDrive - polymtlus\Desktop\POST_DOC\Modules\Regioinvent\src")
import regioinvent

In [10]:
regio = regioinvent.Regioinvent(bw_project_name='metallican', ecoinvent_database_name='cutoff310', ecoinvent_version='3.10')

In [11]:
regio.spatialize_my_ecoinvent()


2025-07-26 13:21:36,314 - Regioinvent - INFO - Creating spatialized biosphere flows...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Title: Writing activities to SQLite3 database:
  Started: 07/26/2025 13:21:37
  Finished: 07/26/2025 13:21:44
  Total time elapsed: 00:00:06
  CPU %: 98.10
  Memory %: 1.34


2025-07-26 13:26:24,034 - Regioinvent - INFO - Extracting ecoinvent to wurst...


Getting activity data


100%|██████████| 23523/23523 [00:00<00:00, 125558.66it/s]


Adding exchange data to activities


100%|██████████| 743409/743409 [00:37<00:00, 19839.63it/s]


Filling out exchange data


100%|██████████| 23523/23523 [00:03<00:00, 6620.87it/s] 
2025-07-26 13:27:12,391 - Regioinvent - INFO - Spatializing ecoinvent...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:46


Title: Writing activities to SQLite3 database:
  Started: 07/26/2025 13:27:14
  Finished: 07/26/2025 13:28:01
  Total time elapsed: 00:00:46
  CPU %: 99.10
  Memory %: 20.05


In [12]:
regio.import_fully_regionalized_impact_method(lcia_method='all')

2025-07-26 13:29:51,106 - Regioinvent - INFO - Importing all available fully regionalized lcia methods for ecoinvent3.10.


In [13]:
regio.regionalize_ecoinvent_with_trade(trade_database_path=r'C:\Users\mp_ma\OneDrive - polymtlus\Desktop\POST_DOC\Modules\Regioinvent\trade_data.db',
                                       regioinvent_database_name='Regioinvent',
                                       cutoff=0.99)

2025-07-26 13:31:19,018 - Regioinvent - INFO - Extracting and formatting trade data...
2025-07-26 13:31:35,057 - Regioinvent - INFO - Regionalizing main inputs of internationally-traded products of ecoinvent...
100%|██████████| 1982/1982 [16:07<00:00,  2.05it/s] 
2025-07-26 13:47:43,050 - Regioinvent - INFO - Regionalizing main inputs of non-internationally traded processes of ecoinvent...
100%|██████████| 67/67 [24:04<00:00, 21.56s/it]
2025-07-26 14:11:47,692 - Regioinvent - INFO - Creating consumption markets for internationally-traded products...
100%|██████████| 1982/1982 [1:17:18<00:00,  2.34s/it]
2025-07-26 15:29:06,868 - Regioinvent - INFO - Link regioinvent processes to each other...
2025-07-26 15:29:45,294 - Regioinvent - INFO - Aggregate duplicates together...
2025-07-26 15:30:00,322 - Regioinvent - INFO - Regionalizing the elementary flows of the regioinvent database...
2025-07-26 15:30:07,453 - Regioinvent - INFO - Write regioinvent database to brightway...
Writing activiti

Title: Writing activities to SQLite3 database:
  Started: 07/26/2025 15:30:16
  Finished: 07/26/2025 15:38:14
  Total time elapsed: 00:07:57
  CPU %: 99.30
  Memory %: 30.04


2025-07-26 15:53:25,648 - Regioinvent - INFO - Connecting ecoinvent to regioinvent processes...


# Custom LCIs

In [40]:
lci_brgm_path = r'not_in_github/Lai_et_al'
lci_files = [
    "Ag_LCIs_BW_Final_v2.xlsx",
    "Al_LCIs_BW_Final_v1.xlsx",
    "Au_LCIs_BW_Final_v1.xlsx",
    "Cd_LCIs_BW_Final_v1.xlsx",
    "Cu_LCIs_BW_Final_v2.xlsx",
    "Fe_LCIs_BW_Final_v1.xlsx",
    "Mo_LCIs_BW_Final_v1.xlsx",
    "Ni_LCIs_BW_Final_v1.xlsx",
    "Te_LCIs_BW_Final_v2.xlsx",
]

In [42]:
for filename in lci_files:
    filepath = os.path.join(lci_brgm_path, filename)
    print(f"Importing: {filename}")

    importer = bw.ExcelImporter(filepath)
    importer.apply_strategies()

    importer.match_database("ecoinvent-3.10-cutoff", fields=("name", "reference product", "unit", "location"))
    importer.match_database("biosphere3", fields=("name", "unit", "categories"))

    unlinked = list(importer.unlinked)
    print(f"  → Unlinked exchanges: {len(unlinked)}")

    if not unlinked:
        importer.write_database()
        print(f"  → Database '{importer}' written.")
    else:
        print(f"  → Skipping '{filename}' due to unlinked exchanges.")


Importing: Ag_LCIs_BW_Final_v2.xlsx
Extracted 5 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 7.24 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


  → Unlinked exchanges: 0


0% [###########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/21/2025 12:58:39
  Finished: 07/21/2025 12:58:39
  Total time elapsed: 00:00:00
  CPU %: 133.00
  Memory %: 1.47
Created database: Silver
  → Database '<bw2io.importers.excel.ExcelImporter object at 0x000002589306A500>' written.
Importing: Al_LCIs_BW_Final_v1.xlsx
Extracted 14 worksheets in 0.36 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_ze

...

Title: Writing activities to SQLite3 database:
  Started: 07/21/2025 13:00:17
  Finished: 07/21/2025 13:00:17
  Total time elapsed: 00:00:00
  CPU %: 89.30
  Memory %: 1.49
Created database: Nickel
  → Database '<bw2io.importers.excel.ExcelImporter object at 0x00000258841D5AB0>' written.
Importing: Te_LCIs_BW_Final_v2.xlsx
Extracted 3 worksheets in 0.11 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zero

Writing activities to SQLite3 database:


  → Unlinked exchanges: 0


0% [#########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/21/2025 13:00:32
  Finished: 07/21/2025 13:00:32
  Total time elapsed: 00:00:00
  CPU %: 151.20
  Memory %: 1.48
Created database: Tellurium
  → Database '<bw2io.importers.excel.ExcelImporter object at 0x000002588336AB00>' written.
